### Creation of the Family Layer

In [248]:
import pandas as pd
from itertools import combinations
import numpy as np

In [232]:
## read in the data

##i could merge on famID 


registry_data_one_year = pd.read_csv("testdata.csv")

In [233]:
registry_data_one_year

,Unnamed: 0,Sun2000niva_old,SUN2000Niva,SUN2000Inr,SUN2000Grp,Barn0_3,Barn4_6,Barn7_10,Barn11_15,Barn16_17,Barn18plus,Barn18_19,Barn20plus,PersonNr,famID,FodesleAr,Alder
0,0,15895,300,761d,Gymnasial utbildning med inriktning mot barn o...,0,0,0,0,0,0,0,0,19370429-1669,19350916-1755,1937,53
1,1,81607,5,840d,Transportutbildning på eftergymnasial nivå ...,0,0,1,1,1,0,0,0,19380301-8616,19350916-1755,1938,52
2,2,70251,300,811d,Hotell- och restaurangutbildning på gymnasial ...,1,2,0,2,2,2,0,0,19350916-1755,19350916-1755,1935,55
3,3,20859,5,762a,Social service- och omsorgsutbildning ...,2,0,2,1,0,0,0,0,19311224-1125,19311224-1125,1931,58
4,4,15015,532,106k,Minst 30 högskolepoäng inom lant- och skogsbru...,0,0,0,0,0,0,0,0,19530115-9550,19311224-1125,1953,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,51220,522,101c,Minst 30 högskolepoäng inom pedagogik och lära...,1,1,1,2,0,0,0,0,19321018-9211,19321018-9211,1932,58
9996,9996,59510,531,312c,Samhälls- och beteendevetenskaplig högskoleutb...,1,0,0,1,0,3,0,0,19290710-2723,19010428-8159,1929,61
9997,9997,79185,557,724c,Tandläkarutbildning ...,0,0,2,0,1,0,0,0,19010428-8159,19010428-8159,1901,89
9998,9998,72465,557,146f,Yrkeslärarutbildning ...,0,0,0,1,0,0,0,0,19390318-1701,19300628-6725,1939,51


In [234]:
#basic step --> connect everyone in the same family:
# Iterate through unique values in 'ConnectionColumn'


#
connections = pd.DataFrame()
for family in registry_data_one_year['famID'].unique():
    # Filter rows where 'famID' has the current value
    subset = registry_data_one_year[registry_data_one_year['famID'] == family]
    
    # Create pairs of IDs
    pairs = pd.merge(subset[['PersonNr']], subset[['PersonNr']], how='cross')
    
    # Add the connection type column
    pairs['ConnectionType'] = "family"
    
    # Append to the connections DataFrame
    connections = pd.concat([connections, pairs])

In [235]:
connections

,PersonNr_x,PersonNr_y,ConnectionType
0,19370429-1669,19370429-1669,family
1,19370429-1669,19380301-8616,family
2,19370429-1669,19350916-1755,family
3,19380301-8616,19370429-1669,family
4,19380301-8616,19380301-8616,family
...,...,...,...
3,19010428-8159,19010428-8159,family
0,19390318-1701,19390318-1701,family
1,19390318-1701,19300628-6725,family
2,19300628-6725,19390318-1701,family


In [282]:
def find_parent_child_relationship(subset):
    parent_relationships = []
    parents = subset[subset['Barn18plus'] > 0]
    children = subset[subset['Alder'] >= 18]
    for _, parent in parents.iterrows():
            age_difference = parent["Alder"] - children["Alder"]
            eligible_children = children[age_difference > 15]
            if not eligible_children.empty:
            
            
            # Assuming "Kön" is a column indicating gender
            #gender = 'mother' if parent['Kön'] == 'female' else 'father'
            
                # Create DataFrame for each parent-child relationship
                child_data = {'personNr1': eligible_children["PersonNr"].tolist(),
                              'personNr2': parent["PersonNr"],
                              'connection': "child of"}

                parent_data = {'personNr1': parent["PersonNr"],
                               'personNr2': eligible_children["PersonNr"].tolist(),
                               'connection': "parent of"}

                parent_relationships.extend([pd.DataFrame(child_data), pd.DataFrame(parent_data)])

    if parent_relationships:
        # Concatenate all DataFrames in the list
        parent_relationships = pd.concat(parent_relationships, ignore_index=True)
    else:
        # Create an empty DataFrame
        parent_relationships = pd.DataFrame(columns=['personNr1', 'personNr2', 'connection'])
    return parent_relationships


In [239]:
def find_relationships(subset):
    #should i change approach here? 
    couple_relationships = pd.DataFrame(columns=['personNr1', 'personNr2', 'connection'])
    selected = subset[['PersonNr', 'Alder']]
    for i in range(len(selected)-1):
        for j in range(i+1, len(selected)):

            person1 = selected.iat[i, 0]
            person2 = selected.iat[j, 0]
            # Calculate age difference
            age_difference = abs(selected.iat[i, 1] - selected.iat[j, 1])
        
            # Check if age difference is less than or equal to 15 years
            
            
            #add check for amount of children (?)
            if age_difference <= 13:
                row_data = pd.DataFrame([{'personNr1':person1 , 'personNr2': person2, 'connection': "partner of"}])
                couple_relationships = pd.concat([couple_relationships, row_data])
               
    return couple_relationships

In [294]:
num_rows = 30

# Create a DataFrame with random numbers in 'personnr1' and 'personnr2'
test_case_grandparent = pd.DataFrame({
    'personNr1': np.random.randint(11, 40, size=num_rows),
    'personNr2': np.random.randint(0, 33, size=num_rows),
})

# Introduce some matches between 'personnr1' and 'personnr2'
num_matches = 3
match_indices = np.random.choice(num_rows, size=num_matches, replace=False)
test_case_grandparent.loc[match_indices, 'personNr2'] = test_case_grandparent.loc[match_indices, 'personNr1']

# Create a 'connection' column with values 'parent of' or 'sibling'
test_case_grandparent['connection'] = np.random.choice(['parent of'], size=num_rows)
test_case_grandparent = test_case_grandparent[test_case_grandparent['personNr1'] != test_case_grandparent['personNr2']]


outcome_2 = find_cousines(test_case_grandparent)
print(outcome_2)

Empty DataFrame
Columns: [personNr1, personNr2, connection]
Index: []


In [255]:
def find_siblings(family_connections):
        # Assuming connections is your DataFrame
    selection = family_connections[family_connections["connection"] == "parent of"]

    # Find all the siblings (people with the same parent)
    result = selection.groupby('personNr1')['personNr2'].agg(list).reset_index()
    result_filtered = result[result['personNr2'].apply(len) > 1]
    
    # Create a list to store DataFrames
    dfs_to_concat = []

    for _, row in result_filtered.iterrows():
        personNr2_list = row['personNr2']
        # Check the length of the list
        if len(personNr2_list) == 2:
            # If length is 2, create a single row with 'siblings' as the connection
            row_data = pd.DataFrame([{'personNr1': personNr2_list[0], 'personNr2': personNr2_list[1], 'connection': 'siblings'}])
            dfs_to_concat.append(row_data)
        elif len(personNr2_list) > 2:
            # If length is greater than 2, create rows for all combinations
            for pair in combinations(personNr2_list, 2):
                row_data = pd.DataFrame([{'personNr1': pair[0], 'personNr2': pair[1], 'connection': 'siblings'}])
                dfs_to_concat.append(row_data)
    
    #drop rows which are the same, currently wouldnt notice them in different order --> can be improved
    dfs_to_concat = pd.concat(dfs_to_concat).drop_duplicates(ignore_index=True)
    
    
    return dfs_to_concat

In [311]:
def find_grandparents_aunts(family_connections):
    
    #make this without warnings --> change append to concat 
    grandparents = pd.DataFrame(columns=['personNr1', 'personNr2', 'connection'])
    parent_rows = family_connections[family_connections['connection'] == 'parent of']
    sibling_rows = family_connections[family_connections['connection'] == 'siblings']
    for _, parent_row in parent_rows.iterrows():
        # Find rows where 'Personnr1' in the original dataframe matches 'personNr2' in the current row
        matching_rows = parent_rows[parent_rows['personNr2'] == parent_row['personNr1']]
        
        matching_rows_auntsuncles = sibling_rows[(sibling_rows['personNr2'] == parent_row['personNr1']) |(sibling_rows['personNr1'] == parent_row['personNr1'])]
        # Iterate through matching rows and add to the result dataframe
        for _, matching_row in matching_rows.iterrows():
            grandparents = grandparents.append({
                'personNr1': matching_rows['personNr1'],
                'personNr2': parent_row['personNr2'],
                'connection': "grandparent of"
            }, ignore_index=True)
            
            grandparents = grandparents.append({
                'personNr1': parent_row['personNr2'],
                'personNr2': matching_rows['personNr1'],
                'connection': "grandchild of"
            }, ignore_index=True)
            
    #aunts and uncles
            grandparents = grandparents.append({
                'personNr1': matching_rows_auntsuncles['personNr1'],
                'personNr2': parent_row['personNr2'],
                'connection': "aunt/uncle of"
            }, ignore_index=True)
            
            grandparents = grandparents.append({
                'personNr1': parent_row['personNr2'],
                'personNr2': matching_rows_auntsuncles['personNr1'],
                'connection': "niece/newphew of"
            }, ignore_index=True)

    return grandparents
            
    

In [312]:
def find_cousines(family_connections):
    cousines= pd.DataFrame(columns=['personNr1', 'personNr2', 'connection'])
    aunt_uncle_rows = family_connections[family_connections['connection'] == 'aunt/uncle of']
    parent_rows = family_connections[family_connections['connection'] == 'parent of']
    
    aunt_uncle_rows = aunt_uncle_rows.reset_index(drop=True)
    parent_rows = parent_rows.reset_index(drop=True)
    for _, aunt_uncle_row in aunt_uncle_rows.iterrows():
        matching_rows = parent_rows[parent_rows['personNr1'] == aunt_uncle_row['personNr1']]
        for _, matching_row in matching_rows.iterrows():
                        
            cousines = cousines.append({
                'personNr1': parent_row['personNr2'],
                'personNr2': matching_row['personNr2'],
                'connection': "cousines"
            }, ignore_index=True)
            
    return cousines

In [313]:
parents = pd.DataFrame()
registry_100 = registry_data_one_year.head(100)



#next step
#create functions to call for creation of children
#create function to call for creation of partners that work for the subset that append to a dictionary(?)

#create dicitionaries and then create a final dataframe for efficiency --> overwork all the steps so far
def create_family_layer(registry_data):
    connections = pd.DataFrame(columns=['personNr1', 'personNr2', 'connection'])
    for family in registry_data['famID'].unique():
        # Filter rows where 'famID' has the current value
        subset = registry_data[registry_data['famID'] == family]
        # Check if each row has a value greater than or equal to 0 in the 'your_column' column
        if len(subset)> 1: 
            #will there be an option to check which year, we are working on

            #work on child-parent relationship 
            #parents = subset[(subset['Barn18plus'] > 0) | (subset['Barn16_17'] >0 ) | (subset['Barn18_19'] > 0) | (subset['Barn20plus'] >0 )].any(axis=1)
            parents = subset[(subset['Barn18plus']>0) |  (subset['Barn16_17'] >0 )  | (subset['Barn18_19'] > 0) | (subset['Barn20plus'] >0 )]
            #print(parents)
            if len(parents)> 1:
                output = find_parent_child_relationship(subset)

                connections = pd.concat([connections, output])

            #work on the partner relationship
            relationships = find_relationships(subset)
            connections = pd.concat([connections, relationships])
            
            
        #assumption --> partners have minimum one child of the same age --> double check with famId creation about partners
        # also connected if both have no children (marriage?)
        
    #sibling relationship
    dfs_to_concat = find_siblings(connections)
    connections = pd.concat([connections] + [dfs_to_concat], ignore_index=True)

    #grandparent/ aunts/uncles / niece .. /cousin relationship

 
    grandparents_connections = find_grandparents_aunts(connections)
    connections = pd.concat([connections] + [grandparents_connections], ignore_index=True)


    #now add siblings to the connection network
    #why so weird format
    #cousine_connections = find_cousines(connections)
    #connections = pd.concat([connections] + [cousine_connections], ignore_index=True)
    

    
    return connections

network = create_family_layer(registry_data_one_year)

C:\Users\Inga\AppData\Local\Temp\ipykernel_3464\2943039749.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grandparents = grandparents.append({
C:\Users\Inga\AppData\Local\Temp\ipykernel_3464\2943039749.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grandparents = grandparents.append({
C:\Users\Inga\AppData\Local\Temp\ipykernel_3464\2943039749.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grandparents = grandparents.append({
C:\Users\Inga\AppData\Local\Temp\ipykernel_3464\2943039749.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grandparents = grandparents.append({
C:\Users\Inga\AppData\Local\Temp\ipykernel_3464\

C:\Users\Inga\AppData\Local\Temp\ipykernel_3464\2943039749.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grandparents = grandparents.append({
C:\Users\Inga\AppData\Local\Temp\ipykernel_3464\2943039749.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grandparents = grandparents.append({
C:\Users\Inga\AppData\Local\Temp\ipykernel_3464\2943039749.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grandparents = grandparents.append({
C:\Users\Inga\AppData\Local\Temp\ipykernel_3464\2943039749.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grandparents = grandparents.append({
C:\Users\Inga\AppData\Local\Temp\ipykernel_3464\

C:\Users\Inga\AppData\Local\Temp\ipykernel_3464\2943039749.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grandparents = grandparents.append({
C:\Users\Inga\AppData\Local\Temp\ipykernel_3464\2943039749.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grandparents = grandparents.append({
C:\Users\Inga\AppData\Local\Temp\ipykernel_3464\2943039749.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grandparents = grandparents.append({
C:\Users\Inga\AppData\Local\Temp\ipykernel_3464\2943039749.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grandparents = grandparents.append({
C:\Users\Inga\AppData\Local\Temp\ipykernel_3464\

In [316]:
network

,personNr1,personNr2,connection
0,19370429-1669,19380301-8616,partner of
1,19370429-1669,19350916-1755,partner of
2,19380301-8616,19350916-1755,partner of
3,19500327-2187,19440411-8885,partner of
4,19340330-9841,19440411-8885,partner of
...,...,...,...
5712,19550515-3421,"5401 19550515-3421 Name: personNr1, dtype: ...",niece/newphew of
5713,"5072 19331027-3995 Name: personNr1, dtype: ...",19720826-9957,grandparent of
5714,19720826-9957,"5072 19331027-3995 Name: personNr1, dtype: ...",grandchild of
5715,"5570 19720826-9957 Name: personNr1, dtype: ...",19720826-9957,aunt/uncle of


In [315]:
#basic analysis of network 